In [1]:
from dicom_csv import join_tree
from pydicom import dcmread
from pathlib import Path

In [ ]:
237646it 

In [ ]:
df_all = join_tree('/mount/attic/data/burdenko-tcia-test/', verbose=1, ignore_extensions=['.zip'])

237646it [1:20:28, 43.62it/s] 

In [8]:
df_all.shape

(344003, 324)

In [9]:
# df_all.to_csv('/mount/attic/data/burdenko-tcia-test/meta.csv')

In [2]:
subject_folder = Path('/mount/attic/data/burdenko-tcia-test/Burdenko-GBM-001/05-06-2004-NA-Radiotherapy planning 00-01357')
df = join_tree(subject_folder, verbose=1)

687it [00:02, 274.07it/s]


In [4]:
df.Modality.value_counts()

MR          484
CT          198
RTSTRUCT      3
RTDOSE        1
RTPLAN        1
Name: Modality, dtype: int64

In [5]:
def get_series_instance_uid(rtstruct_path: [str, Path]):
    """Get SeriesInstanceUID of a reference image of input RTSTRUCT."""
    dicom = dcmread(rtstruct_path)
    return dicom.ReferencedFrameOfReferenceSequence[0]\
                .RTReferencedStudySequence[0]\
                .RTReferencedSeriesSequence[0].SeriesInstanceUID

# from burdenko_glioma_progression import get_series_instance_uid

1. Prepare images:
    - Convert dicoms to nifti-s
2. Prepare masks:
    - Select corresponding image 
    - Extract contours from RTSTRUCTs
    - Convert contours to masks
    - Save masks as dicoms
    - Convert masks to nifit-s
3. (Optionally) Register (rigidly) all images to the same space

# 1. Prepare Images

In [4]:
from burdenko_glioma.nifti import dicom_to_nifti

In [5]:
...

Ellipsis

# 2. Prepare masks



- Select corresponding image
- Extract contours from RTSTRUCTs
- Convert contours to masks
- Save masks as dicoms
- Convert masks to nifit-s


### Select corresponding image

For baseline studies sometimes we have multiple RTSTRUCTs as they were imported from the planning system.
They will typically contain intersected set of ROIs, but correspond to different series within the study.

We will use structures from the RTSTRUCT corresponding to the `CT study`.

In [6]:
computed_tomography_uid = df[df.Modality == "CT"].SeriesInstanceUID.unique()[0]

for _, row in df[df.Modality=="RTSTRUCT"][['PathToFolder','FileName']].iterrows():
    rtstruct = dcmread(subject_folder / row.PathToFolder / row.FileName)
    rtstruct_reference_uid = get_series_instance_uid(subject_folder / row.PathToFolder / row.FileName)
    print(rtstruct_reference_uid)
    if rtstruct_reference_uid == computed_tomography_uid:
        break

1.3.6.1.4.1.14519.5.2.1.222790934420370886390186339781673290484
1.3.6.1.4.1.14519.5.2.1.312435035079316311979018226338735661578
1.3.6.1.4.1.14519.5.2.1.20417500794672593138065669875767558964


### Extract contours from RTSTRUCTs

First, we check which contours are contained in the RTSTRUCT

In [7]:
from dicom_csv.rtstruct import get_contour_seq_name

In [8]:
list(get_contour_seq_name(rtstruct))

['BODY',
 'Brain',
 'BrainStem',
 'Chiasm',
 'Eye_L',
 'Eye_R',
 'Lens_L',
 'Lens_R',
 'OpticNerve_L',
 'OpticNerve_R',
 'GTV_FLAIR',
 'CTV_20 mm',
 'PTV',
 'Brain-PTV',
 'FU_5_FLAIR',
 'FU_5_CE',
 'FU_4_FLAIR',
 'FU_4_CE',
 'FU_3_FLAIR',
 'FU_3_CE',
 'FU_2_FLAIR',
 'FU_2_CE',
 'FU_1_FLAIR',
 'FU_1_CE']

This particular RTSTRUCT contains contours for the baseline visit, and also some contours for control visits (starting with `FU_` for "follow up")

Contours in RTSTRUCTs are stored in a form of coordinates in patient coordinate space, see https://dicom.innolitics.com/ciods/rt-structure-set/roi-contour/30060039/30060040/30060050 for details. Therefore we need to use a corresponding image (again, we will be using CT image).

In [9]:
# read CT series

ct_series = [dcmread(subject_folder / ds.PathToFolder / ds.FileName) \
             for _,ds in df[df.SeriesInstanceUID == computed_tomography_uid].iterrows()]

In [28]:
df.groupby('Modality')['SeriesInstanceUID'].unique()['MR']

array(['1.3.6.1.4.1.14519.5.2.1.312435035079316311979018226338735661578',
       '1.3.6.1.4.1.14519.5.2.1.263118267922629269364621298007167845066',
       '1.3.6.1.4.1.14519.5.2.1.287411424933128087668330241991384428157',
       '1.3.6.1.4.1.14519.5.2.1.222790934420370886390186339781673290484'],
      dtype=object)

In [30]:
mri_uid = '1.3.6.1.4.1.14519.5.2.1.312435035079316311979018226338735661578'

mri_series = [dcmread(subject_folder / ds.PathToFolder / ds.FileName) \
             for _,ds in df[df.SeriesInstanceUID == mri_uid].iterrows()]

image_mri = stack_images(order_series(mri_series))
image_mri.shape

(256, 256, 150)

In [ ]:
ct_series = [dcmread(subject_folder / ds.PathToFolder / ds.FileName) \
             for _,ds in df[df.SeriesInstanceUID == computed_tomography_uid].iterrows()]

In [10]:
from dicom_csv import  order_series, stack_images
from dicom_csv.rtstruct import contours_to_image

In [11]:
len(ct_series)

198

In [12]:
image = stack_images(order_series(ct_series))
image.shape

(512, 512, 198)

In [13]:
contours = contours_to_image(ct_series, rtstruct)

In [14]:
contours.keys()

dict_keys([('BODY', 0), ('Brain', 1), ('BrainStem', 2), ('Chiasm', 3), ('Eye_L', 4), ('Eye_R', 5), ('Lens_L', 6), ('Lens_R', 7), ('OpticNerve_L', 8), ('OpticNerve_R', 9), ('GTV_FLAIR', 10), ('CTV_20 mm', 11), ('PTV', 12), ('Brain-PTV', 13), ('FU_2_FLAIR', 20), ('FU_2_CE', 21), ('FU_1_FLAIR', 22), ('FU_1_CE', 23)])

### Convert contours to masks

contours to mask is already implemented in `dicom_csv`, let's see the results for couple of contours

In [15]:
body_mask = contours[('BODY', 0)].get_mask()
brain_mask = contours[('Brain', 1)].get_mask()
gtv_mask = contours[('GTV_FLAIR', 10)].get_mask()

In [16]:
body_mask.shape, brain_mask.shape, gtv_mask.shape

((512, 512, 198), (512, 512, 198), (512, 512, 198))

Recall, that masks have the same shape as CT image (as expected). Let's read the image assess how masks and image aligns.

In [17]:
from dpipe.im.visualize import slice3d

In [18]:
slice3d(image, body_mask, brain_mask, gtv_mask)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=197), Output()), _dom…

We can clip CT images to a brain window, see https://radiopaedia.org/articles/windowing-ct?lang=gb for details

In [19]:
import numpy as np

In [20]:
slice3d(np.clip(image, 0, 80), body_mask, brain_mask, gtv_mask)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=197), Output()), _dom…

 CT image is not the best modality to check brain tumour, so we can check the correctness of the alignment by masks of the eyes. Recall, that left and right are vice versa on the image, since slices numeration are from the top of the head (0), to the bottom of the chin (n).

In [21]:
left_eye = contours[('Eye_L', 4)].get_mask()
right_eye = contours[('Eye_R', 5)].get_mask()

In [22]:
# Check out slices from 81 to 101

np.nonzero(left_eye.sum(axis=(0,1)))

(array([ 81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
         94,  95,  96,  97,  98,  99, 100, 101]),)

In [23]:
slice3d(
    np.clip(image, 0, 80)+left_eye*100,
    left_eye+right_eye
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=197), Output()), _dom…

In [25]:
slice3d(
    np.clip(image, 0, 80)+gtv_mask*100,
    np.clip(image, 0, 80)
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=197), Output()), _dom…

In [31]:
slice3d(image_mri)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=149), Output()), _dom…

# Save masks as DICOMs

You can already save images and masks as numpy arrays and do your Deep Learning magic, but if you want to use NIFTI format, you will need to first save masks to DICOMs and then run `dcm2niix` on these DICOMs.

There is probably a better way to do this (save masks and images in nifti-s), however back in a days I did not figure how to save masks to niftis to align with the results of `dcm2niix` (ran on images) hence, this crutch proposed by Talgat Saparov.

In [26]:
def mask_to_dicom(series, mask, folder):
    """Writes three-dimensional binary mask to series of reference DICOM files and save them to folder."""
    dtype = series[0].pixel_array.dtype
    for frame, m2 in zip(series, np.moveaxis(mask, 2, 0)):
        byte_mask = m2.astype(dtype).tobytes()
        uid = frame.SOPInstanceUID
        if len(byte_mask) % 2 != 0:
            byte_mask += b'\x00'
        frame.PixelData = byte_mask
        frame.save_as(folder / f'{uid}.dcm')